# Creating a Diffusers Demo with Gradio

In [2]:
# %pip install transformers gradio diffusers

## Example 1: Using the pipeline

In [10]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
from diffusers import StableDiffusionPipeline

model_id = "CompVis/stable-diffusion-v1-4"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, revision="fp16", torch_dtype=torch.float16
).to(device)

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

In [11]:
prompt = "a photograph of an astronaut riding a horse"

outputs = pipe(prompt)
outputs

  0%|          | 0/51 [00:00<?, ?it/s]

StableDiffusionPipelineOutput(images=[<PIL.Image.Image image mode=RGB size=512x512 at 0x7F673911B0D0>], nsfw_content_detected=[False])

In [7]:
def predict(prompt):
    return pipe(prompt).images[0]

In [9]:
import gradio as gr

In [ ]:
gradio_ui = gr.Interface(
    fn=predict,
    title="Stable Diffusion Demo",
    description="Enter a description of an image you'd like to generate!",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Image(),
    ],
    examples=[
        ["an astronaut riding a horse"]
    ],
)

gradio_ui.launch(debug=True)

/home/lewis_huggingface_co/miniconda3/envs/hf/lib/python3.9/site-packages/gradio/inputs.py:26: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/home/lewis_huggingface_co/miniconda3/envs/hf/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/lewis_huggingface_co/miniconda3/envs/hf/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/home/lewis_huggingface_co/miniconda3/envs/hf/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/home/lewis_huggingface_co/miniconda3/envs/hf/lib/python3.9/site-packages/gradio/outputs.py:42: UserWarning: Usage of gradio.outputs is deprecated,

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Example 2: Using the Inference API from the Hugging Face Hub

In [ ]:
from huggingface_hub import InferenceApi

text = "My favourite book is Cryptonomicon!"
inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
preds = inference(inputs=text)
preds[0]

In [ ]:
sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True) 
sorted_preds

In [ ]:
def inference_predict(text):
    inference = InferenceApi("lewtun/xlm-roberta-base-finetuned-marc-en")
    preds = inference(inputs=text)
    sorted_preds = sorted(preds[0], key=lambda d: d['score'], reverse=True)[0]
    return label2emoji[sorted_preds["label"]], round(sorted_preds["score"], 5)

In [ ]:
inference_predict(text)

In [ ]:
gradio_ui = gr.Interface.load(
    name="lewtun/xlm-roberta-base-finetuned-marc",
    src="huggingface",
    fn=inference_predict,
    title="Review analysis",
    description="Enter some text and check if model detects it's star rating.",
    inputs=[
        gr.inputs.Textbox(lines=5, label="Paste some text here"),
    ],
    outputs=[
        gr.outputs.Textbox(label="Label"),
        gr.outputs.Textbox(label="Score"),
    ],
    examples=[
        ["My favourite book is Cryptonomicon!"], ["私の好きな本は「クリプトノミコン」です"]
    ],
)

gradio_ui.launch(debug=True)